In [1]:
# delete this cell if working on Pycharm
!pip install Bio
!pip install import-ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 5.1 MB/s 
     |████████████████████████████████| 2.3 MB 55.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import os
# so we can import utils notebook (delete if working on Pycharm), you might need to change it to your working directory path

from google.colab import drive 
drive.mount('/content/drive/')
!ls
%cd "drive"
%cd "MyDrive"
%cd "hackathonPrivate"
import import_ipynb
import utils


Mounted at /content/drive/
drive  sample_data
/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/hackathonPrivate
importing Jupyter notebook from utils.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
###############################################################################
#                                                                             #
#              Parameters you can change, but don't have to                   #
#                                                                             #
###############################################################################


# number of ResNet blocks for the first ResNet and the kernel size.
RESNET_1_BLOCKS = 3 
RESNET_1_KERNEL_SIZE = 15
RESNET_1_KERNEL_NUM = 64


###############################################################################
#                                                                             #
#                        Parameters you need to choose                        #
#                                                                             #
###############################################################################


# number of ResNet blocks for the second ResNet, dilation list to repeat and the kernel size.

RESNET_2_BLOCKS = 3
RESNET_2_KERNEL_SIZE = 3
RESNET_2_KERNEL_NUM = 32
DILATION = [1, 2, 4, 8]

# percentage of dropout for the dropout layer
DROPOUT = 0.15

# number of epochs, Learning rate and Batch size
EPOCHS = 60
LR = 0.01
BATCH = 32

In [4]:
def resnet_1(input_layer, res1_blocks, res1_ker_size, res1_ker_num, activation_res1):  # TODO: implement this!
    """
    ResNet layer - input -> BatchNormalization -> Conv1D -> Relu -> BatchNormalization -> Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    for _ in range(res1_blocks):
      out_batchnorm = layers.BatchNormalization()(input_layer)
      out_conv1 = layers.Conv1D(res1_ker_num, res1_ker_size, padding='same', activation=activation_res1)(out_batchnorm)
      out_batchnorm = layers.BatchNormalization()(out_conv1)
      out_conv1 = layers.Conv1D(res1_ker_num, res1_ker_size, padding='same', activation=activation_res1)(out_batchnorm)
      input_layer = layers.Add()([input_layer, out_conv1])
    return input_layer





In [5]:
def resnet_2(input_layer, dilaitons, res2_blocks, res2_ker_size, res2_ker_num, activation_res2):  # TODO: implement this!
    """
    Dilated ResNet layer - input -> BatchNormalization -> dilated Conv1D -> Relu -> BatchNormalization -> dilated Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    for _ in range(res2_blocks):
      for dilation in dilaitons:
        out_batchnorm = layers.BatchNormalization()(input_layer)
        out_conv1 = layers.Conv1D(res2_ker_num, res2_ker_size, padding='same', activation=activation_res2, dilation_rate=dilation)(out_batchnorm)
        out_batchnorm = layers.BatchNormalization()(out_conv1)
        out_conv1 = layers.Conv1D(res2_ker_num, res2_ker_size, padding='same', activation=activation_res2, dilation_rate=dilation)(out_batchnorm)
        input_layer = layers.Add()([input_layer, out_conv1])
    return input_layer

In [22]:
def build_network(res1_blocks = 3, res1_ker_size = 15, res1_ker_num = 64, activation_res1 = 'relu',
                  res2_blocks = 3, res2_ker_size = 3, res2_ker_num = 32, activation_res2 = 'relu',
                  dilaitons = [1, 2, 4, 8], dropout = 0.15, activation_dropout = 'elu'):
    """
    builds the neural network architecture as shown in the exercise.
    :return: a Keras Model
    """
    ...
    # input, shape (NB_MAX_LENGTH,FEATURE_NUM)
    input_layer = tf.keras.Input(shape=(utils.NB_MAX_LENGTH, utils.FEATURE_NUM))

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(res1_ker_num, res1_ker_size, padding='same')(input_layer)

    # first ResNet -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    resnet_layer = resnet_1(conv1d_layer, res1_blocks, res1_ker_size, res1_ker_num, activation_res1)

    # # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(res2_ker_num, res2_ker_size, padding="same")(resnet_layer)

    # # second ResNet -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    resnet_layer = resnet_2(conv1d_layer, dilaitons, res2_blocks, res2_ker_size, res2_ker_num, activation_res2)

    dropout_layer = layers.Dropout(dropout)(resnet_layer)

    conv1d_layer = layers.Conv1D(np.ceil(res2_ker_num/2), res2_ker_size, padding="same", activation=activation_dropout)(dropout_layer)

    output = layers.Dense(utils.OUTPUT_SIZE)(conv1d_layer)


    return tf.keras.Model(input_layer, output, name='from_seq_to_struct')



In [7]:
def plot_val_train_loss(history):
    """
    plots the train and validation loss of the model at each epoch, saves it in 'model_loss_history.png'
    :param history: history object (output of fit function)
    :return: None
    """
    ig, axes = plt.subplots(1, 1, figsize=(15,3))
    axes.plot(history.history['loss'][10:], label='Training loss')
    axes.plot(history.history['val_loss'][10:], label='Validation loss')
    axes.legend()
    axes.set_title("Train and Val MSE loss")

    plt.savefig("/content/drive/MyDrive/hackathonPrivate/model_loss_history")  # TODO: you can change the path here


In [32]:
def train_models(num_models_to_train, meta_parametes):
    '''
    meta params: 0.res1_blocks, 1.res1_ker_size, 2.res1_ker_num, 3.activation_res1,
    4.res2_blocks, 5.res2_ker_size, 6.res2_ker_num, 7.activation_res2,
    8.dilaitons, 9.dropout, 10.epoch, 11.batch, 12.activation_dropout 13.LR
    '''
    path_for_models =  "/content/drive/MyDrive/hackathonPrivate/models"
    X = np.load('/content/drive/MyDrive/hackathonPrivate/files/train_input.npy')
    Y = np.load('/content/drive/MyDrive/hackathonPrivate/files/train_labels.npy')

    X_train, X_val, y_train, y_val  = train_test_split(X, Y, test_size=0.2, random_state=1)
    for i in range(num_models_to_train):
        print("model %d" %i)
        if (meta_parametes[i] == []):
          model = build_network()
          my_optimizer= tf.keras.optimizers.Adam(learning_rate=LR)

        else:
          model = build_network(meta_parametes[i][0], meta_parametes[i][1], meta_parametes[i][2], meta_parametes[i][3], meta_parametes[i][4], 
                                meta_parametes[i][5], meta_parametes[i][6],meta_parametes[i][7], meta_parametes[i][8], meta_parametes[i][9], 
                                meta_parametes[i][12])
          my_optimizer= tf.keras.optimizers.Adam(learning_rate=meta_parametes[i][13])
        model.compile(optimizer=my_optimizer, loss="mse")
        batch = BATCH if meta_parametes[i] == [] else meta_parametes[i][11]
        epoch = EPOCHS if meta_parametes[i] == [] else meta_parametes[i][10]
        history = model.fit(X_train, y_train, epochs=epoch, batch_size=batch, validation_data=(X_val, y_val))
        model.save("%s/model%d.tf" %(path_for_models, i))


In [35]:
def provide_predictions(path_to_PDB):

    NUM_OF_MODELS = 10
    PATH_TO_MODELS = '/content/drive/MyDrive/hackathonPrivate/models'
    predictions  =[]
    protein_to_predict = utils.generate_input(path_to_PDB)
    for i in range(NUM_OF_MODELS):
      model = tf.keras.models.load_model("%s/model%d.tf" %(PATH_TO_MODELS, i))
      prediction = model.predict(protein_to_predict[np.newaxis,:,:])
      predictions.append(prediction)
    return predictions



In [39]:
if __name__ == '__main__':

    #
    #   Activate the train models only if you wish to edit/change/add the models
    #   Edit the meta_parameters to be as you wish in the models  
    #

    # res1_blocks = 3, res1_ker_size = 15, res1_ker_num = 64, activation_res1 = 'relu',
    #                 res2_blocks = 3, res2_ker_size = 3, res2_ker_num = 32, activation_res2 = 'relu',
    #                 dilaitons = [1, 2, 4, 8], dropout = 0.15, epoch = 60, 
    #                 batch =32, activation_dropout = 'elu', LR= 0.01
    meta_parameters = [[],[3, 15, 64, 'relu', 5, 5, 32, 'relu', [1,2,4], 0.2, 50, 32, 'elu', 0.01],
                       [3, 15, 64, 'relu', 3, 5, 24, 'relu', [2,4,8,16], 0.25, 60, 64, 'elu', 0.01],
                       [3, 25, 48, 'relu', 2, 5, 32, 'relu', [1,2,4,8,16], 0.15, 60, 32, 'elu', 0.01],
                       [3, 15, 64, 'relu', 5, 5, 32, 'gelu', [1,2,4,8], 0.2, 60, 32, 'elu', 0.01],
                       [3, 15, 64, 'softplus', 3, 3, 32, 'softplus', [1,2,4,8], 0.15, 60, 32, 'elu', 0.01],
                       [3, 15, 64, 'elu', 5, 3, 64, 'elu', [1,2,4,16], 0.2, 60, 32, 'relu', 0.01],
                       [3, 15, 64, 'relu', 3, 5, 32, 'LeakyReLU', [1,2,4,8], 0.25, 60, 32, 'elu', 0.01], #<----
                       [4, 15, 64, 'silu', 5, 3, 32, 'silu', [1,2,4], 0.2, 50, 64, 'elu', 0.01],
                       [3, 15, 64, 'relu', 2, 5, 64, 'relu', [1,2,4,8,16], 0.25, 120, 32, 'elu', 0.001]]
    train_models(10, meta_parameters)
    predictions = provide_predictions('/content/drive/MyDrive/Hachathon Guy Yaniv Sagi Adi/Data/6xw6/6xw6.pdb')

    # to_predict = utils.generate_input(os.path.join("/content/drive/MyDrive/Hachathon Guy Yaniv Sagi Adi/Data/6xw6", "6xw6.pdb"))
    # prediction = model.predict(to_predict[np.newaxis,:,:])
    # print(prediction)
    # np.save('/content/drive/MyDrive/hackathonPrivate/example.npy', prediction)
    # matrix = utils.matrix_to_pdb(utils.get_seq_aa(os.path.join("/content/drive/MyDrive/hackathonPrivate", "6xw6.pdb"), "H")[0], prediction[0, :, :],"our_6xw6")


model 9
Epoch 1/120
50/50 [==============================] - 9s 58ms/step - loss: 20.8644 - val_loss: 23.1711
Epoch 2/120
50/50 [==============================] - 2s 37ms/step - loss: 4.5260 - val_loss: 12.5538
Epoch 3/120
50/50 [==============================] - 2s 39ms/step - loss: 3.1759 - val_loss: 8.8658
Epoch 4/120
50/50 [==============================] - 2s 38ms/step - loss: 2.6105 - val_loss: 4.8075
Epoch 5/120
50/50 [==============================] - 2s 38ms/step - loss: 2.2233 - val_loss: 3.3386
Epoch 6/120
50/50 [==============================] - 2s 38ms/step - loss: 2.0066 - val_loss: 2.2110
Epoch 7/120
50/50 [==============================] - 2s 39ms/step - loss: 1.8110 - val_loss: 2.3217
Epoch 8/120
50/50 [==============================] - 2s 39ms/step - loss: 1.6938 - val_loss: 1.4805
Epoch 9/120
50/50 [==============================] - 2s 38ms/step - loss: 1.5338 - val_loss: 1.3468
Epoch 10/120
50/50 [==============================] - 2s 38ms/step - loss: 1.4849 - val_l

INFO:tensorflow:Assets written to: /content/drive/MyDrive/hackathonPrivate/models/model9.tf/assets


[array([[[-1.07146444e+01, -1.20924997e+01, -1.04556024e-01, ...,
         -1.18386879e+01, -1.19578743e+01, -7.55763233e-01],
        [-1.00558920e+01, -1.21564302e+01,  1.31395829e+00, ...,
         -9.37727737e+00, -1.22795877e+01,  2.89031792e+00],
        [-7.46187639e+00, -9.50765991e+00,  2.59244609e+00, ...,
         -6.49705315e+00, -8.45133495e+00,  2.13982391e+00],
        ...,
        [-4.47639823e-02, -4.84122336e-02,  3.60617042e-03, ...,
          2.81799734e-02, -8.41288269e-03,  8.96036625e-03],
        [-4.47639823e-02, -4.84122336e-02,  3.60617042e-03, ...,
          2.81799734e-02, -8.41288269e-03,  8.96036625e-03],
        [-4.47639823e-02, -4.84122336e-02,  3.60617042e-03, ...,
          2.81799734e-02, -8.41288269e-03,  8.96036625e-03]]],
      dtype=float32), array([[[-10.702721  , -11.994923  ,   0.07986099, ..., -10.727639  ,
         -10.742992  ,  -0.14245659],
        [ -9.157342  , -10.70139   ,   2.0072777 , ...,  -9.328785  ,
         -11.284063  ,   3.5